In [1]:
HOST = '37.139.42.145'
DBNAME = 'game-analytics'
USER = 'user'
PASSWORD = '5ohJYY4ci8TUJWhQ'
from sqlalchemy import create_engine
engine = create_engine(f'postgresql://{USER}:{PASSWORD}@{HOST}/{DBNAME}')
import pandas as pd
group_schema='marketing_analytics'

facebook=pd.read_csv('./downloads/t4_spend_facebook.csv')
googleads =pd.read_csv('./downloads/t4_spend_googleads.csv')
personaly =pd.read_csv('./downloads/t4_spend_personaly.csv')


In [2]:
#общий расчет из второго задания
ma2 = pd.read_sql(f'''
   
SELECT t.media_source, t.campaign, 
        installs, payers, 
        purchase,rev as revenue, 
        cast(payers as float)/cast(installs as float) as c2p, 
        rev/installs as RPI
        
            FROM (
                    SELECT  count(distinct player_id) as installs, media_source, campaign
                    FROM {group_schema}.t2_installs
                    WHERE status='Non-organic'
                          and is_tester=False 
                          and is_cheater=False 
                          and cast(server_time as date) between '2021-02-01' and '2021-05-09'
                    GROUP BY media_source, campaign) as t
            LEFT JOIN (
                   SELECT count(distinct p.player_id) as payers, count(p.player_id) as purchase, 
                           media_source, campaign, sum(revenue) as rev
                   FROM {group_schema}.t2_purchases as p
                   LEFT JOIN {group_schema}.t2_installs as i
                     ON p.player_id= i.player_id
                   WHERE status='Non-organic'
                         and is_tester=False 
                         and is_cheater=False 
                         and cast(p.server_time as date) between '2021-02-01' and '2021-05-09'
                   GROUP BY media_source, campaign) as r
            ON t.campaign= r.campaign


''', con=engine)

unityads=pd.read_sql(f'''
SELECT  campaign, sum(cost)
    FROM (
        SELECT player_id, cost 
        FROM {group_schema}.t4_spend_unityads 
        )as p

    LEFT JOIN ( SELECT player_id, campaign
                FROM {group_schema}.t2_installs
                WHERE status='Non-organic'
                             and is_tester=False 
                             and is_cheater=False
                             and media_source='unityads_int'
                             and cast(server_time as date) between '2021-02-01' and '2021-05-09'
                        ) as i
                         ON p.player_id= i.player_id
GROUP BY campaign

''', con=engine)

In [3]:
facebook=facebook.groupby(['Campaign Name'], as_index=False).agg({'Amount Spent (USD)':'sum'})
facebook=facebook.rename(columns={'Campaign Name':'campaign','Amount Spent (USD)':'Spend (USD)'})

googleads=googleads.groupby(['Campaign'], as_index=False).agg({'Cost':'sum'})
googleads=googleads.rename(columns={'Campaign':'campaign','Cost':'Spend (USD)'})

personaly=personaly.agg({'Spend':'sum'})


In [4]:
data=pd.concat([facebook, googleads,unityads.rename(columns={'sum':'Spend (USD)'})],
                ignore_index=True, sort=False)
result=pd.merge(ma2, data, on=['campaign'], how='left')
#замена значения
result.at[43,'Spend (USD)']=personaly.iloc[0]

In [5]:
result=result.fillna(0)
result['profit']=result['revenue']-result['Spend (USD)']
result['CPI']=result['Spend (USD)']/result['installs']
result['ROI']=(result['revenue']-result['Spend (USD)'])/result['Spend (USD)']

In [6]:
result=result.fillna(0)

In [9]:
result.to_csv('./downloads/MA4.csv', index=False)